# Łączenie z bazą danych i instrukcja select
## Marcin Bereźnicki
Wykonaj zapytania za pomocą skryptów pythona:
1. Ile kategorii filmów mamy w wypożyczalni?
2. Wyświetl listę kategorii w kolejności alfabetycznej.
3. Znajdź najstarszy i najmłodszy film do wypożyczenia.
4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?
5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?
6. Znajdź największą płatność wypożyczenia.
7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.
8. Gdzie mieszkają członkowie personelu?
9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?
10. Jakie kategorie filmów zostały wypożyczone przez klientów?
11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.
12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

### 1. Ile kategorii filmów mamy w wypożyczalni?

In [2]:
df1 = pd.read_sql("""
SELECT
    DISTINCT name
FROM
    category
""", con=connection)

print('\n Liczba kategorii filmów: {0}'.format(len(df1.index)))
df1


 Liczba kategorii filmów: 16


,name
0,Comedy
1,Drama
2,Horror
3,Sci-Fi
4,Classics
5,Foreign
6,Games
7,New
8,Travel
9,Action


### 2. Wyświetl listę kategorii w kolejności alfabetycznej.

In [3]:
df2 = pd.read_sql("""
SELECT
    DISTINCT name
FROM
    category
ORDER BY 
    name ASC
""", con=connection)
df2

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


### 3. Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [4]:
df3_1 = pd.read_sql("""
SELECT
    title, release_year
FROM
    film
ORDER BY 
    release_year ASC LIMIT 1
""", con=connection)

df3_2 = pd.read_sql("""
SELECT
    title, release_year
FROM
    film
ORDER BY 
    release_year DESC LIMIT 1
""", con=connection)

print('Najstarszy film: {0}, {1} rok'.format(df3_1.iloc[0]['title'], df3_1.iloc[0]['release_year']))
print('Najmłodszy film: {0}, {1} rok'.format(df3_2.iloc[0]['title'], df3_2.iloc[0]['release_year']))

Najstarszy film: Chamber Italian, 2006 rok
Najmłodszy film: Chamber Italian, 2006 rok


### 4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [5]:
df4 = pd.read_sql("""
SELECT
    rental_date
FROM
    rental
WHERE 
    rental_date >= '2005-07-01 00:00:00' AND rental_date <= '2005-08-01 23:59:59'
""", con=connection)

print('\n Liczba wypożyczeń filmów między 2005-07-01 a 2005-08-01: {0}'.format(len(df4.index)))
df4


 Liczba wypożyczeń filmów między 2005-07-01 a 2005-08-01: 7380


,rental_date
0,2005-07-05 22:49:24
1,2005-07-05 22:51:44
2,2005-07-05 22:56:33
3,2005-07-05 22:57:34
4,2005-07-05 22:59:53
...,...
7375,2005-08-01 23:49:33
7376,2005-08-01 23:50:22
7377,2005-08-01 23:51:00
7378,2005-08-01 23:53:45


### 5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [6]:
df5 = pd.read_sql("""
SELECT
    rental_date
FROM
    rental
WHERE 
    rental_date >= '2010-01-01 00:00:00' AND rental_date <= '2011-02-01 23:59:59'
""", con=connection)

print('\n Liczba wypożyczeń filmów między 2005-07-01 a 2005-08-01: {0}'.format(len(df5.index)))
df5


 Liczba wypożyczeń filmów między 2005-07-01 a 2005-08-01: 0


,rental_date


### 6. Znajdź największą płatność wypożyczenia.

In [7]:
df6 = pd.read_sql("""
SELECT
    amount
FROM
    payment
ORDER BY 
    amount DESC LIMIT 1 
""", con=connection)

print('Największa płatność: {0}'.format(df6.iloc[0]['amount']))
df6

Największa płatność: 11.99


,amount
0,11.99


### 7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [8]:
df7 = pd.read_sql("""
SELECT 
    first_name, last_name, country 
FROM 
    customer 
INNER JOIN 
    address ON customer.address_id = address.address_id
INNER JOIN 
    city ON address.city_id = city.city_id 
INNER JOIN 
    country ON city.country_id = country.country_id 
WHERE 
    country='Poland' OR country='Nigeria' OR country='Bangladesh'
""",con=connection)
df7

,first_name,last_name,country
0,Sonia,Gregory,Nigeria
1,Brian,Wyman,Poland
2,Sidney,Burleson,Poland
3,Rodney,Moeller,Nigeria
4,Stephen,Qualls,Bangladesh
5,Velma,Lucas,Nigeria
6,Marilyn,Ross,Nigeria
7,Elsie,Kelley,Nigeria
8,Gladys,Hamilton,Nigeria
9,Frank,Waggoner,Bangladesh


### 8. Gdzie mieszkają członkowie personelu?

In [9]:
df8 = pd.read_sql("""
SELECT 
    first_name, last_name, address, city, country
FROM 
    staff 
INNER JOIN 
    address ON staff.address_id = address.address_id
INNER JOIN 
    city ON address.city_id = city.city_id 
INNER JOIN 
    country ON city.country_id = country.country_id
""",con=connection)
df8

,first_name,last_name,address,city,country
0,Mike,Hillyer,23 Workhaven Lane,Lethbridge,Canada
1,Jon,Stephens,1411 Lillydale Drive,Woodridge,Australia


### 9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [10]:
df9 = pd.read_sql("""
SELECT 
    first_name, last_name, country
FROM 
    staff 
INNER JOIN 
    address ON staff.address_id = address.address_id
INNER JOIN 
    city ON address.city_id = city.city_id 
INNER JOIN 
    country ON city.country_id = country.country_id
WHERE 
    country='Argentina' OR country='Spain'
""",con=connection)
print('Liczba pracowników z Argentyny lub Hiszpanii: {0}'.format(len(df9.index)))
df9

Liczba pracowników z Argentyny lub Hiszpanii: 0


,first_name,last_name,country


### 10. Jakie kategorie filmów zostały wypożyczone przez klientów?

In [11]:
df10 = pd.read_sql("""
SELECT 
    DISTINCT name
FROM 
    rental 
INNER JOIN 
    inventory ON rental.inventory_id = inventory.inventory_id
INNER JOIN 
    film ON inventory.film_id = film.film_id 
INNER JOIN 
    film_category ON film.film_id = film_category.film_id
INNER JOIN 
    category ON film_category.category_id = category.category_id
""",con=connection)
df10

,name
0,Horror
1,Comedy
2,Drama
3,Sci-Fi
4,Foreign
5,Classics
6,Games
7,Travel
8,New
9,Music


### 11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [12]:
df11 = pd.read_sql("""
SELECT 
    DISTINCT name
FROM 
    rental 
INNER JOIN 
    inventory ON rental.inventory_id = inventory.inventory_id
INNER JOIN 
    film ON inventory.film_id = film.film_id 
INNER JOIN 
    film_category ON film.film_id = film_category.film_id
INNER JOIN 
    category ON film_category.category_id = category.category_id
INNER JOIN 
    staff ON rental.staff_id = staff.staff_id
INNER JOIN 
    store ON staff.store_id = store.store_id
INNER JOIN 
    address ON store.address_id = address.address_id
INNER JOIN 
    city ON address.city_id = city.city_id 
INNER JOIN 
    country ON city.country_id = country.country_id
WHERE 
    country='Canada'
""",con=connection)
df11

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


### 12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [13]:
df12 = pd.read_sql("""
SELECT 
    title, first_name, last_name
FROM 
    film 
INNER JOIN 
    film_actor ON film.film_id = film_actor.film_id
INNER JOIN 
    actor ON film_actor.actor_id = actor.actor_id
WHERE 
    first_name='Olympia' AND last_name='Pfeiffer'
    OR first_name='Julia' AND last_name='Zellweger'
    OR first_name='Ellen' AND last_name='Presley'
""",con=connection)
df12

,title,first_name,last_name
0,Bilko Anonymous,Ellen,Presley
1,Caribbean Liberty,Ellen,Presley
2,Casper Dragonfly,Ellen,Presley
3,Empire Malkovich,Ellen,Presley
4,Floats Garden,Ellen,Presley
...,...,...,...
64,Outlaw Hanky,Julia,Zellweger
65,Panky Submarine,Julia,Zellweger
66,Rider Caddyshack,Julia,Zellweger
67,Won Dares,Julia,Zellweger
